In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess, time, os, requests, shutil, zipfile, json, re
from PIL import Image
from google.colab import files

# Start Ollama for any future LLM-based logic
subprocess.Popen(["ollama", "serve"])
time.sleep(5)
!ollama pull qwen2.5-coder:1.5b
print("✅ Environment Ready")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

✅ Environment Ready


In [13]:
import os, shutil, zipfile, json, re, requests
from PIL import Image
from google.colab import files

class BalancedFoodAgent:
    def __init__(self, model="qwen2.5-coder:1.5b"):
        self.url = "http://localhost:11434/api/generate"
        self.model = model
        self.work_dir = "/content/agent_workspace"
        self.project_dir = os.path.join(self.work_dir, "mod_project")
        self.mod_id = ""
        self.package_path = ""
        self.main_class_name = ""
        os.makedirs(self.work_dir, exist_ok=True)

    def setup_project(self, zip_path):
        if os.path.exists(self.project_dir): shutil.rmtree(self.project_dir)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(self.project_dir)
        fabric_json = os.path.join(self.project_dir, "src/main/resources/fabric.mod.json")
        with open(fabric_json, 'r') as f:
            data = json.load(f)
            self.mod_id = data.get('id', 'modid')
            entrypoint = data['entrypoints']['main'][0]
            self.main_class_name = entrypoint.split('.')[-1]
            self.package_path = entrypoint.rsplit('.', 1)[0].replace('.', '/')
        print(f"✅ Loaded Mod: {self.mod_id} | Main: {self.main_class_name}")



    def get_balanced_logic(self, description):
        prompt = f"""[INST] You are a Minecraft Data Bot.
        Provide these values for: {description}
        - nutrition: (number 1-12)
        - saturation: (number 0.1-1.2)
        - effect_id: (e.g., MobEffects.FIRE_RESISTANCE)
        - duration_seconds: (number)
        [/INST]"""

        try:
            response = requests.post(self.url, json={"model": self.model, "prompt": prompt, "stream": False}, timeout=10)
            raw_text = response.json().get('response', '')

            # BRUTE FORCE REGEX EXTRACTION
            # This ignores all brackets, commas, and quotes. It just finds the pattern.
            nutrition = re.search(r'nutrition["\s:]+(\d+)', raw_text)
            saturation = re.search(r'saturation["\s:]+([\d.]+)', raw_text)
            effect = re.search(r'MobEffects\.([A-Z_]+)', raw_text)
            duration = re.search(r'duration["\s_seconds:]+(\d+)', raw_text)

            # Fallback values if the AI fails to provide a specific field
            logic = {
                "nutrition": int(nutrition.group(1)) if nutrition else 6,
                "saturation": float(saturation.group(1)) if saturation else 0.6,
                "effect_id": f"MobEffects.{effect.group(1)}" if effect else "MobEffects.REGENERATION",
                "duration": (int(duration.group(1)) if duration else 10) * 20, # Convert to ticks
                "amplifier": 0
            }

            # Clamp values to prevent unrealistic nutrition/saturation
            logic["nutrition"] = max(1, min(logic["nutrition"], 12))
            logic["saturation"] = max(0.1, min(logic["saturation"], 1.2))

            return logic

        except Exception as e:
            print(f"⚠️ Brute force parser handled an error: {e}. Using defaults.")
            return {"nutrition": 6, "saturation": 0.6, "effect_id": "MobEffects.REGENERATION", "duration": 200, "amplifier": 0}

    def create_mod_files(self, item_id, display_name, ai_data):
        # 1. Texture Upload
        print(f"\n🖼️ Upload texture for {display_name}...")
        tex_path = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}/textures/item/"
        os.makedirs(tex_path, exist_ok=True)
        uploaded = files.upload()
        if uploaded:
            os.rename(list(uploaded.keys())[0], f"{tex_path}{item_id}.png")

        # 2. JSON Resources (1.21.1 Dual-file structure)
        assets = f"{self.project_dir}/src/main/resources/assets/{self.mod_id}"
        os.makedirs(f"{assets}/models/item", exist_ok=True)
        os.makedirs(f"{assets}/items", exist_ok=True)
        os.makedirs(f"{assets}/lang", exist_ok=True)

        with open(f"{assets}/models/item/{item_id}.json", 'w') as f:
            json.dump({"parent": "minecraft:item/generated", "textures": {"layer0": f"{self.mod_id}:item/{item_id}"}}, f, indent=2)
        with open(f"{assets}/items/{item_id}.json", 'w') as f:
            json.dump({"model": {"type": "minecraft:model", "model": f"{self.mod_id}:item/{item_id}"}}, f, indent=2)

        lang_file = f"{assets}/lang/en_us.json"
        lang_data = {}
        if os.path.exists(lang_file):
            with open(lang_file, 'r') as f: lang_data = json.load(f)
        lang_data[f"item.{self.mod_id}.{item_id}"] = display_name
        with open(lang_file, 'w') as f: json.dump(lang_data, f, indent=2)

        # 3. Java Injection (Mojang Mappings + Creative Tab)
        java_dir = f"{self.project_dir}/src/main/java/{self.package_path}"
        package_name = self.package_path.replace('/', '.')

        mod_items_code = f"""package {package_name};

import net.fabricmc.fabric.api.itemgroup.v1.ItemGroupEvents;
import net.minecraft.core.Registry;
import net.minecraft.core.registries.BuiltInRegistries;
import net.minecraft.core.registries.Registries;
import net.minecraft.resources.Identifier;
import net.minecraft.resources.ResourceKey;
import net.minecraft.world.effect.MobEffectInstance;
import net.minecraft.world.effect.MobEffects;
import net.minecraft.world.food.FoodProperties;
import net.minecraft.world.item.CreativeModeTabs;
import net.minecraft.world.item.Item;
import net.minecraft.world.item.component.Consumable;
import net.minecraft.world.item.component.Consumables;
import net.minecraft.world.item.consume_effects.ApplyStatusEffectsConsumeEffect;
import java.util.function.Function;

public class ModItems {{
    public static final FoodProperties {item_id.upper()}_FOOD = new FoodProperties.Builder()
            .nutrition({ai_data['nutrition']})
            .saturationModifier({ai_data['saturation']}f)
            .alwaysEdible().build();

    public static final Consumable {item_id.upper()}_CONSUMABLE = Consumables.defaultFood()
            .onConsume(new ApplyStatusEffectsConsumeEffect(new MobEffectInstance({ai_data['effect_id']}, {ai_data['duration']}, {ai_data['amplifier']}), 1.0f))
            .build();

    public static final Item {item_id.upper()} = register("{item_id}", Item::new,
            new Item.Properties().food({item_id.upper()}_FOOD, {item_id.upper()}_CONSUMABLE));

    public static <GenericItem extends Item> GenericItem register(String name, Function<Item.Properties, GenericItem> itemFactory, Item.Properties settings) {{
        ResourceKey<Item> itemKey = ResourceKey.create(Registries.ITEM, Identifier.fromNamespaceAndPath({self.main_class_name}.MOD_ID, name));
        GenericItem item = itemFactory.apply(settings.setId(itemKey));
        Registry.register(BuiltInRegistries.ITEM, itemKey, item);
        return item;
    }}

    public static void initialize() {{
        ItemGroupEvents.modifyEntriesEvent(CreativeModeTabs.FOOD_AND_DRINKS).register(entries -> {{
            entries.accept({item_id.upper()});
        }});
    }}
}}
"""
        with open(os.path.join(java_dir, "ModItems.java"), 'w') as f:
            f.write(mod_items_code)

        # Inject initialize() hook
        main_file = os.path.join(java_dir, f"{self.main_class_name}.java")
        with open(main_file, 'r') as f: content = f.read()
        if "ModItems.initialize();" not in content:
            content = re.sub(r'(public void onInitialize\(\) \{)', r'\1\n        ModItems.initialize();', content)
            with open(main_file, 'w') as f: f.write(content)

    def finish(self):
        shutil.make_archive("/content/balanced_mod", 'zip', self.project_dir)
        print("🎁 SUCCESS! Your mod is balanced and ready.")
        files.download("/content/balanced_mod.zip")

agent = BalancedFoodAgent()

In [14]:
# 1. Start the process
uploaded = files.upload()
agent.setup_project(list(uploaded.keys())[0])

# 2. Run with the Brute Force logic
description = "A spicy taco that gives Levitation for 30 seconds"
item_id = "spicy_taco"
item_name = "Spicy Taco"

logic = agent.get_balanced_logic(description)
print(f"📊 Extracted Logic: {logic}")

agent.create_mod_files(item_id, item_name, logic)
agent.finish()

Saving balanced_mod.zip to balanced_mod (1).zip
✅ Loaded Mod: taco | Main: Taco
📊 Extracted Logic: {'nutrition': 6, 'saturation': 0.6, 'effect_id': 'MobEffects.LEVITATION', 'duration': 200, 'amplifier': 0}

🖼️ Upload texture for Spicy Taco...


Saving vegetable_corn.png to vegetable_corn.png
🎁 SUCCESS! Your mod is balanced and ready.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Install Java 21
!apt-get install openjdk-21-jdk-headless -qq > /dev/null

# Set the environment variables so Gradle uses Java 21
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
!java -version

openjdk version "21.0.9" 2025-10-21
OpenJDK Runtime Environment (build 21.0.9+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 21.0.9+10-Ubuntu-122.04, mixed mode, sharing)


In [15]:
import os
import subprocess

# 1. Ensure the Gradlew script is executable
gradle_path = os.path.join(agent.project_dir, "gradlew")
!chmod +x {gradle_path}

# 2. Run the Build
print("🔨 Building Mod... This may take 1-2 minutes for the first run.")
try:
    # We use --no-daemon for Colab to save memory
    result = subprocess.run(
        [gradle_path, "build", "--no-daemon"],
        cwd=agent.project_dir,
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        print("✅ Build Successful!")

        # 3. Locate the generated .jar
        libs_dir = os.path.join(agent.project_dir, "build/libs")
        if os.path.exists(libs_dir):
            jars = [f for f in os.listdir(libs_dir) if f.endswith(".jar") and "-dev" not in f and "-sources" not in f]
            if jars:
                final_jar = os.path.join(libs_dir, jars[0])
                print(f"📦 Found Mod File: {jars[0]}")
                files.download(final_jar)
            else:
                print("❌ Could not find the final .jar file in build/libs.")
    else:
        print("❌ Build Failed! Check the errors below:")
        print(result.stdout)
        print(result.stderr)

except Exception as e:
    print(f"⚠️ An error occurred during the build process: {e}")

🔨 Building Mod... This may take 1-2 minutes for the first run.
✅ Build Successful!
📦 Found Mod File: taco-1.0.0.jar


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>